# Payment Gateway

Designing a robust payment gateway involves several critical components, from understanding the high-level architecture to choosing the right payment providers and implementing secure, maintainable code. Below, I'll address each of your requirements in detail:

## TOC:
* [1. Identify Location based on payment](#find-location-while-pay)
* [2. Database Schema](#db_schema)
* [3. ER Diagram](#er_diagram)
* [3. High-Level Flow Diagram for Payment Gateway with Razorpay and Stripe](#high-level-design)
* [4. UPI integration](#upi_integration)

## 1. Identify Location based on payment <a class="anchor" id="find-location-while-pay">

To determine the country of a user when they make a payment from a website, you can use various methods and libraries in Python. Here are some approaches:

### 1. IP Geolocation Services
<b>IP Geolocation API:</b> You can use a geolocation service that provides an API to map the user's IP address to their location, including the country.

<b>Libraries:</b><br>
<b>GeoIP2 (MaxMind):</b><br>
This library can be used with MaxMind’s GeoIP2 databases or web services.<br>
<b>ipinfo:</b><br>
A simple API service that provides IP geolocation data. You can use their Python library to easily fetch the country.<br>
<b>ipstack:</b><br>
Another API service that provides detailed location data based on the IP address.<br>
<b>geocoder:</b><br>
A Python library that supports multiple geocoding services, including IP geolocation.

### 2. Payment Gateway APIs
<b>Stripe:</b><br>
Stripe’s API can provide the country of the cardholder based on the card’s BIN (Bank Identification Number).<br>
<b>Razorpay:</b><br>
Razorpay might also provide country information in the payment details or metadata, depending on their API implementation.<br>
<b>PayPal:</b><br>
PayPal's API includes the country in the billing information of the user.

### 3. Custom User Input
You can also ask the user to select their country during checkout and store this information in your system.

### Example using GeoIP2:

In [ ]:
import geoip2.database

# Load the GeoLite2 Country database (you can download this from MaxMind)
reader = geoip2.database.Reader('/path/to/GeoLite2-Country.mmdb')

def get_country_from_ip(ip_address):
    response = reader.country(ip_address)
    return response.country.name, response.country.iso_code

# Example IP address
ip_address = '128.101.101.101'
country_name, country_code = get_country_from_ip(ip_address)
print(f'Country: {country_name}, Code: {country_code}')


### Example using ipinfo:

In [ ]:
import ipinfo

access_token = 'your_ipinfo_access_token'
handler = ipinfo.getHandler(access_token)

def get_country_from_ip(ip_address):
    details = handler.getDetails(ip_address)
    return details.country, details.country_name

# Example IP address
ip_address = '128.101.101.101'
country_code, country_name = get_country_from_ip(ip_address)
print(f'Country: {country_name}, Code: {country_code}')


# DataBase Schema <a class="anchor" id="db_schema">

## 1. Number of Tables
The number of tables can vary depending on the complexity of the system, but a typical setup might include the following:

<b>Users Table:</b><br> Stores user-related information.<br>
<b>Payment Methods Table:</b><br> Stores different payment methods used by users.<br>
<b>Transactions Table:</b><br> Records each payment transaction.<br>
<b>Currencies Table:</b><br> Stores currency details.<br>
<b>Payment Status Table:</b><br> Tracks the status of each payment.<br>
<b>Payment Gateway Table:</b><br> Contains details about the payment gateway (e.g., Stripe, Razorpay).<br>
<b>Order Table:</b><br> Links payments to orders.<br>
<b>Audit Logs Table:</b><br> Stores logs for tracking changes or operations related to payments.<br>
<b>Country Table:</b><br> Stores country details to link with transactions or users.<br>
<b>Product Table:</b><br> If relevant, stores product information related to the transaction.

## 2. Table Structures and Useful Information
<h3>Users Table</h3>

In [ ]:
CREATE TABLE Users (
    user_id SERIAL PRIMARY KEY,
    username VARCHAR(100) UNIQUE NOT NULL,
    email VARCHAR(255) UNIQUE NOT NULL,
    created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP NULL
);


<h3>Payment Methods Table</h3>

In [ ]:
CREATE TABLE PaymentMethods (
    method_id SERIAL PRIMARY KEY,
    method_name VARCHAR(50) NOT NULL,
    method_details JSONB NOT NULL
);


<h3>Transactions Table</h3>

In [ ]:
CREATE TABLE Transactions (
    transaction_id SERIAL PRIMARY KEY,
    user_id INT NOT NULL REFERENCES Users(user_id),
    order_id INT NOT NULL REFERENCES Orders(order_id),
    method_id INT NOT NULL REFERENCES PaymentMethods(method_id),
    amount DECIMAL(10, 2) NOT NULL,
    currency_code CHAR(3) NOT NULL REFERENCES Currencies(currency_code),
    status_id INT NOT NULL REFERENCES PaymentStatus(status_id),
    transaction_date TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    gateway_id INT NOT NULL REFERENCES PaymentGateway(gateway_id),
    country_id INT NOT NULL REFERENCES Country(country_id),
    created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP NULL
);


<h3>Currencies Table</h3>

In [ ]:
CREATE TABLE Currencies (
    currency_code CHAR(3) PRIMARY KEY,
    currency_name VARCHAR(50) NOT NULL,
    symbol VARCHAR(5) NOT NULL
);


<h3>Payment Status Table</h3>

In [ ]:
CREATE TABLE PaymentStatus (
    status_id SERIAL PRIMARY KEY,
    status_name VARCHAR(50) UNIQUE NOT NULL,
    description TEXT
);


### Status Values
<h3>Successful:</h3> The payment is in the range between PIN_PYMT_SUCCESS and PIN_PYMT_SUSPENSE.<br>
<h3>Suspended:</h3> The payment is in the range between PIN_PYMT_SUSPENSE and PIN_PYMT_FAILED.<br>
<h3>Failed:</h3> The payment is in the range between PIN_PYMT_FAILED and PIN_PYMT_STATUS_MAX.


### Descriptions
<h3>Positive:</h3> Payments that are received, accepted, or accepted with change, irrespective of their numeric range.<br>
<h3>Pending:</h3> Payments awaiting further processing, irrespective of their numeric range.<br>
<h3>Negative:</h3> Payments that have been rejected, irrespective of their numeric range.<br>
<h3>Positive and Negative:</h3> Indicates partial acceptance, showing a combination of both positive and negative aspects.<br>
<h3>Paid:</h3> Payments completed successfully.<br>
<h3>Unpaid:</h3> Payments processed with errors or incompleteness.<br>
<h3>Void:</h3> Payments marked as void but eligible for reissue.<br>
<h3>Canceled:</h3> Payments or items canceled and no longer valid.<br>
This table should provide a comprehensive overview of the payment statuses, including their values, numeric ranges, and descriptions.

<h3>Payment Gateway Table</h3>

In [ ]:
CREATE TABLE PaymentGateway (
    gateway_id SERIAL PRIMARY KEY,
    gateway_name VARCHAR(100) UNIQUE NOT NULL,
    gateway_details JSONB NOT NULL
);


<h3>Order Table</h3>

In [ ]:
CREATE TABLE Orders (
    order_id SERIAL PRIMARY KEY,
    user_id INT NOT NULL REFERENCES Users(user_id),
    order_date TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    total_amount DECIMAL(10, 2) NOT NULL,
    currency_code CHAR(3) NOT NULL REFERENCES Currencies(currency_code),
    status_id INT NOT NULL REFERENCES PaymentStatus(status_id)
);


<h3>Audit Logs Table</h3>

In [ ]:
CREATE TABLE AuditLogs (
    log_id SERIAL PRIMARY KEY,
    table_name VARCHAR(50) NOT NULL,
    operation_type VARCHAR(10) NOT NULL,
    operation_time TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    old_data JSONB,
    new_data JSONB,
    user_id INT NOT NULL REFERENCES Users(user_id)
);


<h3>Country Table</h3>

In [ ]:
CREATE TABLE Country (
    country_id SERIAL PRIMARY KEY,
    country_name VARCHAR(100) NOT NULL,
    country_code CHAR(2) NOT NULL UNIQUE
);


In [ ]:
# list of tables create in postgreSQL

SMART_PAY_USERS
SMART_PAY_ORDERS
SMART_PAY_PAYMENT_METHODS
SMART_PAY_AUDIT_LOGS
SMART_PAY_TRANSACTIONS
SMART_PAY_CURRENCIES
SMART_PAY_PAYMENT_STATUS
SMART_PAY_PAYMENT_GATEWAY
SMART_PAY_COUNTRY

# ER Diagram<a class="anchor" id="er_diagram"></a>

To create an ER (Entity-Relationship) diagram based on the above table structure, the diagram would include the following entities and their relationships:

### Entities:
Users<br>
Payment Methods<br>
Transactions<br>
Currencies<br>
Payment Status<br>
Payment Gateway<br>
Orders<br>
Audit Logs<br>
Country<br>

### Relationships:
`Users` are related to `Transactions` and `Orders` (One-to-Many).<br>
`Payment Methods` are related to `Transactions` (One-to-Many).<br>
`Transactions` are related to `Currencies`, `Payment Status`, `Payment Gateway`, and `Country` (Many-to-One).<br>
`Orders` are related to `Transactions` (One-to-Many).<br>
`Audit Logs` are related to `Users` (Many-to-One).<br>

### ER Diagram Components:

`Entities`: Represented by rectangles.
`Attributes`: Represented by ovals connected to their respective entities.
`Relationships`: Represented by diamonds connecting the entities.

<img src="smart-pay-er-diagram.PNG" alt="Girl in a jacket" width="900" height="900">

## 2. High-Level Flow Diagram for Payment Gateway with Razorpay and Stripe <a class="anchor" id="high-level-design"></a>
Here's a high-level overview of how a payment gateway can integrate both Razorpay and Stripe to handle payments in India (INR) and USD:

In [ ]:
+----------------+          +-------------------+          +----------------+
|                |          |                   |          |                |
|  User/Client   | <------> |  Your Application | <------> | Payment Gateway|
|                |          |                   |          |  (Razorpay &   |
+----------------+          +-------------------+          |    Stripe)     |
                                                              +-------+--------+
                                                                      |
                                                                      |
                                                                      v
                                                              +-------+--------+
                                                              |                |
                                                              |  Payment APIs  |
                                                              | (Razorpay API, |
                                                              |  Stripe API)   |
                                                              +-------+--------+
                                                                      |
                                                                      |
                                                                      v
                                                              +-------+--------+
                                                              |                |
                                                              |   Payment       |
                                                              |   Processing    |
                                                              | (Banks, Cards,  |
                                                              |  Wallets, etc.) |
                                                              +-----------------+


## Flow Steps:

### User Interaction:

The user initiates a payment from your application (e.g., e-commerce checkout).
### Your Application:

The application captures payment details and decides whether to use Razorpay or Stripe based on criteria like currency (INR/USD), user location, etc.
### Payment Gateway Integration:

The application communicates with the selected payment provider's API (Razorpay or Stripe) to process the payment.
### Payment Processing:

The payment provider handles the transaction with banks, card networks, wallets, etc., and returns the transaction status to your application.
Confirmation:

Your application updates the user with the payment status and proceeds accordingly (e.g., order confirmation).

## 2. Razorpay vs. Stripe: Best Payment Gateway for India and USD Payments
When choosing between Razorpay and Stripe for handling payments in India and USD, consider the following factors:

### Razorpay
### Pros:

### Local Focus:
Tailored for the Indian market with extensive support for INR transactions.
Payment Methods: Supports a wide range of Indian payment methods like UPI, NetBanking, wallets (e.g., Paytm, PhonePe), and credit/debit cards.
Ease of Integration: Comprehensive documentation and plugins for popular platforms.
Regulatory Compliance: Compliant with Indian financial regulations, which can simplify legal aspects.
Customer Support: Strong support for Indian businesses with localized assistance.
### Cons:

### International Support: 
While Razorpay has expanded internationally, its primary strength lies in the Indian market.
Currency Support: Limited support for currencies outside India, though it does handle some international transactions.
Stripe
Pros:

### Global Reach: 
Excellent for handling multiple currencies, including USD, EUR, and others.
Advanced Features: Offers a suite of advanced tools like subscription management, fraud detection (Radar), and more.
Developer-Friendly: Highly customizable with robust APIs and extensive documentation.
Scalability: Suitable for businesses of all sizes, from startups to large enterprises.
Cons:

### Local Support in India: 
Historically, Stripe's support in India has been limited compared to Razorpay, though it has been expanding.
Payment Methods: Primarily focuses on global payment methods, which might not cover all popular Indian methods like UPI out-of-the-box.
Recommendation
For India and USD Payments:

### Primary Gateway: Razorpay
Why: If your primary market is India and you need robust support for INR transactions and local payment methods, Razorpay is the better choice.

### Supplementary Gateway: Stripe
Why: To handle USD and other international transactions efficiently, integrating Stripe alongside Razorpay can provide comprehensive coverage.
Alternative Suggestion:

<h3>PayPal:</h3> Another global payment gateway that supports multiple currencies, including USD and INR, but it's generally considered more expensive and less developer-friendly compared to Stripe and Razorpay.
Final Decision:

<h3>Use Both Razorpay and Stripe: </h3>
This hybrid approach allows you to leverage Razorpay's strength in the Indian market and Stripe's global capabilities, ensuring flexibility and comprehensive coverage for both INR and USD payments.


To rewrite the implementation with appropriate design patterns, we'll focus on leveraging patterns like the Factory Pattern for gateway selection, Strategy Pattern for handling different payment processes, and Template Method Pattern for shared payment flow logic. This approach will make the code more modular, maintainable, and scalable.

## Implementation with Design Patterns
### 1. Project Structure

In [ ]:
payment_gateway/
├── gateways/
│   ├── __init__.py
│   ├── base.py
│   ├── razorpay_gateway.py
│   ├── stripe_gateway.py
├── factories/
│   ├── __init__.py
│   ├── gateway_factory.py
├── strategies/
│   ├── __init__.py
│   ├── payment_strategy.py
├── config.py
├── main.py
├── requirements.txt
└── .env


## 2. Design Patterns Implementation
### 2.1. Base Payment Gateway Interface (gateways/base.py)
This remains the abstract base class, ensuring consistency across payment gateway implementations. We'll also use the Template Method Pattern here to define the payment flow skeleton.

In [ ]:
# gateways/base.py

from abc import ABC, abstractmethod

class PaymentGateway(ABC):
    def process_payment(self, amount, currency, receipt, **kwargs):
        payment = self.create_payment(amount, currency, receipt, **kwargs)
        self.verify_payment(payment['id'], payment['signature'], **kwargs)
        self.on_payment_success(payment)
    
    @abstractmethod
    def create_payment(self, amount, currency, receipt, **kwargs):
        pass

    @abstractmethod
    def verify_payment(self, payment_id, signature, **kwargs):
        pass
    
    def on_payment_success(self, payment):
        print(f"Payment successful: {payment}")


### 2.2. Razorpay Integration (gateways/razorpay_gateway.py)

In [ ]:
# gateways/razorpay_gateway.py

import razorpay
from .base import PaymentGateway
from config import Config

class RazorpayGateway(PaymentGateway):
    def __init__(self):
        self.client = razorpay.Client(auth=(Config.RAZORPAY_KEY, Config.RAZORPAY_SECRET))

    def create_payment(self, amount, currency, receipt, **kwargs):
        payment = self.client.order.create({
            'amount': int(amount * 100),  # Razorpay expects amount in paise
            'currency': currency,
            'receipt': receipt,
            'payment_capture': '1'
        })
        return payment

    def verify_payment(self, payment_id, signature, **kwargs):
        try:
            params_dict = {
                'razorpay_order_id': kwargs.get('order_id'),
                'razorpay_payment_id': payment_id,
                'razorpay_signature': signature
            }
            status = self.client.utility.verify_payment_signature(params_dict)
            return status
        except razorpay.errors.SignatureVerificationError:
            return False


### 2.3. Stripe Integration (gateways/stripe_gateway.py)

In [ ]:
# gateways/stripe_gateway.py

import stripe
from .base import PaymentGateway
from config import Config

class StripeGateway(PaymentGateway):
    def __init__(self):
        stripe.api_key = Config.STRIPE_API_KEY

    def create_payment(self, amount, currency, receipt, **kwargs):
        payment_intent = stripe.PaymentIntent.create(
            amount=int(amount * 100),  # Stripe expects amount in cents
            currency=currency,
            receipt_email=kwargs.get('receipt_email'),
            metadata={'receipt': receipt}
        )
        return payment_intent

    def verify_payment(self, payment_id, **kwargs):
        try:
            payment_intent = stripe.PaymentIntent.retrieve(payment_id)
            return payment_intent.status == 'succeeded'
        except stripe.error.StripeError:
            return False


## 2.4. Payment Gateway Factory (factories/gateway_factory.py)
Use the Factory Pattern to create instances of the appropriate payment gateway based on the input.

In [ ]:
# factories/gateway_factory.py

from gateways.razorpay_gateway import RazorpayGateway
from gateways.stripe_gateway import StripeGateway

class PaymentGatewayFactory:
    @staticmethod
    def get_gateway(gateway_name):
        if gateway_name.lower() == 'razorpay':
            return RazorpayGateway()
        elif gateway_name.lower() == 'stripe':
            return StripeGateway()
        else:
            raise ValueError("Unsupported payment gateway")


## 2.5. Payment Strategy (strategies/payment_strategy.py)
Implement the Strategy Pattern to encapsulate different payment processing strategies.

In [ ]:
# strategies/payment_strategy.py

from factories.gateway_factory import PaymentGatewayFactory

class PaymentContext:
    def __init__(self, gateway_name):
        self.gateway = PaymentGatewayFactory.get_gateway(gateway_name)
    
    def execute_payment(self, amount, currency, receipt, **kwargs):
        self.gateway.process_payment(amount, currency, receipt, **kwargs)


## 2.6. Main Application Logic (main.py)
The main script uses the PaymentContext to process payments via different gateways using the strategy pattern.

In [ ]:
# main.py

from strategies.payment_strategy import PaymentContext

if __name__ == "__main__":
    # Example usage

    # Process Razorpay Payment
    razorpay_context = PaymentContext('razorpay')
    razorpay_context.execute_payment(
        amount=500.00,  # INR
        currency='INR',
        receipt='order_rcptid_11'
    )

    # Process Stripe Payment
    stripe_context = PaymentContext('stripe')
    stripe_context.execute_payment(
        amount=20.00,  # USD
        currency='usd',
        receipt='order_rcptid_22',
        receipt_email='customer@example.com'
    )


## 3. Summary of Design Patterns Used
### Factory Pattern (PaymentGatewayFactory): 
Centralized object creation for different payment gateways based on input.
### Strategy Pattern (PaymentContext): 
Encapsulates the payment process and allows easy switching between different payment methods (Razorpay, Stripe, etc.).
### Template Method Pattern (PaymentGateway): 
Provides a blueprint for payment processing, allowing specific gateways to implement their logic while ensuring a consistent workflow.
## 4. Security Considerations
Sensitive data is still managed securely using environment variables (config.py).
The modular design and usage of design patterns ensure that the application is extendable, maintainable, and secure.
This pattern-driven approach makes your payment gateway system more organized and easier to extend or modify, such as adding new payment gateways or altering the payment flow.

****************************************

********************************

To test the payment gateway implementation thoroughly, you can use Python's unittest framework, combined with unittest.mock to mock external API calls. Below is a set of robust test cases to validate the functionality of the payment gateway system.

## 1. Setting Up the Test Environment
First, let's structure the test suite. Create a tests/ directory in your project with the following files:

In [ ]:
tests/
├── __init__.py
├── test_payment_gateway.py
└── test_gateway_factory.py

## 2. Test Cases for Payment Gateway Functionality
### 2.1. Test the Razorpay and Stripe Gateways (test_payment_gateway.py)
This test file covers the functionality of the individual payment gateways, ensuring that payments are created and verified correctly.

In [ ]:
import unittest
from unittest.mock import patch, MagicMock
from gateways.razorpay_gateway import RazorpayGateway
from gateways.stripe_gateway import StripeGateway

class TestRazorpayGateway(unittest.TestCase):

    @patch('razorpay.Client')
    def test_create_payment(self, MockClient):
        mock_client = MockClient.return_value
        mock_client.order.create.return_value = {
            'id': 'order_1',
            'amount': 50000,
            'currency': 'INR',
            'receipt': 'order_rcptid_11'
        }

        gateway = RazorpayGateway()
        payment = gateway.create_payment(amount=500.00, currency='INR', receipt='order_rcptid_11')

        mock_client.order.create.assert_called_once_with({
            'amount': 50000,  # Razorpay expects amount in paise
            'currency': 'INR',
            'receipt': 'order_rcptid_11',
            'payment_capture': '1'
        })
        self.assertEqual(payment['id'], 'order_1')

    @patch('razorpay.Client')
    def test_verify_payment(self, MockClient):
        mock_client = MockClient.return_value
        mock_client.utility.verify_payment_signature.return_value = True

        gateway = RazorpayGateway()
        status = gateway.verify_payment(payment_id='pay_1', signature='sig_1', order_id='order_1')

        mock_client.utility.verify_payment_signature.assert_called_once()
        self.assertTrue(status)

class TestStripeGateway(unittest.TestCase):

    @patch('stripe.PaymentIntent')
    def test_create_payment(self, MockPaymentIntent):
        mock_payment_intent = MockPaymentIntent.create.return_value
        mock_payment_intent.id = 'pi_1'

        gateway = StripeGateway()
        payment = gateway.create_payment(amount=20.00, currency='usd', receipt='order_rcptid_22')

        MockPaymentIntent.create.assert_called_once_with(
            amount=2000,  # Stripe expects amount in cents
            currency='usd',
            receipt_email=None,
            metadata={'receipt': 'order_rcptid_22'}
        )
        self.assertEqual(payment.id, 'pi_1')

    @patch('stripe.PaymentIntent')
    def test_verify_payment(self, MockPaymentIntent):
        mock_payment_intent = MockPaymentIntent.retrieve.return_value
        mock_payment_intent.status = 'succeeded'

        gateway = StripeGateway()
        status = gateway.verify_payment(payment_id='pi_1')

        MockPaymentIntent.retrieve.assert_called_once_with('pi_1')
        self.assertTrue(status)

if __name__ == '__main__':
    unittest.main()


## Explanation:

Mocking External Services: We use unittest.mock.patch to mock the Razorpay and Stripe client methods. This prevents actual API calls during testing.
Assertions: We assert that the methods are called with the correct arguments and that the returned values match expected results.

## 2.2. Test the Payment Gateway Factory (test_gateway_factory.py)
This test file ensures that the PaymentGatewayFactory returns the correct gateway based on the input.

In [ ]:
import unittest
from factories.gateway_factory import PaymentGatewayFactory
from gateways.razorpay_gateway import RazorpayGateway
from gateways.stripe_gateway import StripeGateway

class TestPaymentGatewayFactory(unittest.TestCase):

    def test_get_razorpay_gateway(self):
        gateway = PaymentGatewayFactory.get_gateway('razorpay')
        self.assertIsInstance(gateway, RazorpayGateway)

    def test_get_stripe_gateway(self):
        gateway = PaymentGatewayFactory.get_gateway('stripe')
        self.assertIsInstance(gateway, StripeGateway)

    def test_invalid_gateway(self):
        with self.assertRaises(ValueError):
            PaymentGatewayFactory.get_gateway('invalid_gateway')

if __name__ == '__main__':
    unittest.main()


### Explanation:

Factory Pattern: These tests check that the factory correctly creates instances of RazorpayGateway and StripeGateway and raises an exception for unsupported gateways.
## 3. Running the Tests
To run the tests, execute the following command from the root of your project:

In [ ]:
python -m unittest discover tests

## 4. Additional Test Cases
### 4.1. Testing the Payment Context Strategy (test_payment_strategy.py)
This test ensures that the PaymentContext correctly delegates the payment processing to the appropriate gateway.

In [ ]:
import unittest
from unittest.mock import patch
from strategies.payment_strategy import PaymentContext

class TestPaymentContext(unittest.TestCase):

    @patch('strategies.payment_strategy.PaymentGatewayFactory.get_gateway')
    def test_execute_payment_with_razorpay(self, mock_get_gateway):
        mock_gateway = mock_get_gateway.return_value
        mock_gateway.process_payment = MagicMock()

        context = PaymentContext('razorpay')
        context.execute_payment(amount=500.00, currency='INR', receipt='order_rcptid_11')

        mock_gateway.process_payment.assert_called_once_with(
            amount=500.00,
            currency='INR',
            receipt='order_rcptid_11'
        )

    @patch('strategies.payment_strategy.PaymentGatewayFactory.get_gateway')
    def test_execute_payment_with_stripe(self, mock_get_gateway):
        mock_gateway = mock_get_gateway.return_value
        mock_gateway.process_payment = MagicMock()

        context = PaymentContext('stripe')
        context.execute_payment(amount=20.00, currency='usd', receipt='order_rcptid_22')

        mock_gateway.process_payment.assert_called_once_with(
            amount=20.00,
            currency='usd',
            receipt='order_rcptid_22',
            receipt_email=None
        )

if __name__ == '__main__':
    unittest.main()


### Explanation:

Strategy Pattern: These tests ensure that PaymentContext correctly selects the gateway and executes the payment using the chosen gateway.
## 5. Summary of Test Coverage
Gateway Functionality: Tests for Razorpay and Stripe gateways ensure that payments are created and verified correctly.
Factory: Tests for the factory pattern ensure correct instantiation of gateways based on input.
Strategy: Tests for the strategy pattern verify that the payment context correctly handles different payment strategies.
These test cases will provide comprehensive coverage for the payment gateway implementation, ensuring that each component functions as expected and integrates well with others.

*******************************************

# UPI Integration<a class="anchor" id="upi_integration"></a>

***************************

In [ ]:
+----------------+          +-------------------+          +----------------+
|                |          |                   |          |                |
|  User/Client   | <------> |  Your Application | <------> | Payment Gateway|
|                |          |                   |          |  (Razorpay,    |
+----------------+          +-------------------+          |   Stripe, UPI) |
                                                              +-------+--------+
                                                                      |
                                                                      |
                                                                      v
                                                              +-------+--------+
                                                              |                |
                                                              |  Payment APIs  |
                                                              | (Razorpay API, |
                                                              |  Stripe API,   |
                                                              |   UPI API)     |
                                                              +-------+--------+
                                                                      |
                                                                      |
                                                                      v
                                                              +-------+--------+
                                                              |                |
                                                              |  Payment       |
                                                              |  Processing    |
                                                              | (Banks, Cards, |
                                                              | Wallets, UPI,  |
                                                              | etc.)          |
                                                              +----------------+


In [ ]:
payment_gateway/
├── gateways/
│   ├── __init__.py
│   ├── base.py
│   ├── razorpay_gateway.py
│   ├── stripe_gateway.py
│   ├── upi_gateway.py  # New UPI gateway
├── factories/
│   ├── __init__.py
│   ├── gateway_factory.py
├── strategies/
│   ├── __init__.py
│   ├── payment_strategy.py
├── config.py
├── main.py
├── requirements.txt
└── .env


## 3. Python Code Implementation
### 3.1. UPI Payment Gateway (gateways/upi_gateway.py)

In [ ]:
# gateways/upi_gateway.py

from .base import PaymentGateway

class UpiGateway(PaymentGateway):
    def __init__(self):
        # Initialization for UPI (could include API keys, etc.)
        pass

    def create_payment(self, amount, currency, receipt, **kwargs):
        # Simulate a UPI payment creation
        payment = {
            'id': 'upi_order_1',
            'amount': amount,
            'currency': currency,
            'receipt': receipt
        }
        return payment

    def verify_payment(self, payment_id, **kwargs):
        # Simulate UPI payment verification
        return True


## 3.2. Update the Payment Gateway Factory (factories/gateway_factory.py)
### Update the factory to include UPI:

In [ ]:
# factories/gateway_factory.py

from gateways.razorpay_gateway import RazorpayGateway
from gateways.stripe_gateway import StripeGateway
from gateways.upi_gateway import UpiGateway  # Import UPI gateway

class PaymentGatewayFactory:
    @staticmethod
    def get_gateway(gateway_name):
        if gateway_name.lower() == 'razorpay':
            return RazorpayGateway()
        elif gateway_name.lower() == 'stripe':
            return StripeGateway()
        elif gateway_name.lower() == 'upi':  # UPI case
            return UpiGateway()
        else:
            raise ValueError("Unsupported payment gateway")


## 3.3. Update Main Application Logic (main.py)
## Update the main script to handle UPI payments:

In [ ]:
# main.py

from strategies.payment_strategy import PaymentContext

if __name__ == "__main__":
    # Example usage

    # Process Razorpay Payment
    razorpay_context = PaymentContext('razorpay')
    razorpay_context.execute_payment(
        amount=500.00,  # INR
        currency='INR',
        receipt='order_rcptid_11'
    )

    # Process Stripe Payment
    stripe_context = PaymentContext('stripe')
    stripe_context.execute_payment(
        amount=20.00,  # USD
        currency='usd',
        receipt='order_rcptid_22',
        receipt_email='customer@example.com'
    )

    # Process UPI Payment
    upi_context = PaymentContext('upi')
    upi_context.execute_payment(
        amount=1000.00,  # INR
        currency='INR',
        receipt='order_rcptid_33'
    )


## 4. Robust Test Cases
### 4.1. Test UPI Gateway (test_payment_gateway.py)
Add test cases for the UPI gateway:

In [ ]:
import unittest
from unittest.mock import patch, MagicMock
from gateways.razorpay_gateway import RazorpayGateway
from gateways.stripe_gateway import StripeGateway
from gateways.upi_gateway import UpiGateway  # Import UPI Gateway

class TestUpiGateway(unittest.TestCase):

    def test_create_payment(self):
        gateway = UpiGateway()
        payment = gateway.create_payment(amount=1000.00, currency='INR', receipt='order_rcptid_33')

        self.assertEqual(payment['id'], 'upi_order_1')
        self.assertEqual(payment['amount'], 1000.00)
        self.assertEqual(payment['currency'], 'INR')
        self.assertEqual(payment['receipt'], 'order_rcptid_33')

    def test_verify_payment(self):
        gateway = UpiGateway()
        status = gateway.verify_payment(payment_id='upi_order_1')

        self.assertTrue(status)

if __name__ == '__main__':
    unittest.main()


### 4.2. Update Payment Gateway Factory Tests (test_gateway_factory.py)
Add test cases for the UPI gateway in the factory:

In [ ]:
import unittest
from factories.gateway_factory import PaymentGatewayFactory
from gateways.razorpay_gateway import RazorpayGateway
from gateways.stripe_gateway import StripeGateway
from gateways.upi_gateway import UpiGateway  # Import UPI Gateway

class TestPaymentGatewayFactory(unittest.TestCase):

    def test_get_razorpay_gateway(self):
        gateway = PaymentGatewayFactory.get_gateway('razorpay')
        self.assertIsInstance(gateway, RazorpayGateway)

    def test_get_stripe_gateway(self):
        gateway = PaymentGatewayFactory.get_gateway('stripe')
        self.assertIsInstance(gateway, StripeGateway)

    def test_get_upi_gateway(self):
        gateway = PaymentGatewayFactory.get_gateway('upi')
        self.assertIsInstance(gateway, UpiGateway)

    def test_invalid_gateway(self):
        with self.assertRaises(ValueError):
            PaymentGatewayFactory.get_gateway('invalid_gateway')

if __name__ == '__main__':
    unittest.main()


### 4.3. Update Payment Context Strategy Tests (test_payment_strategy.py)
Add test cases for UPI in the strategy:

In [ ]:
import unittest
from unittest.mock import patch
from strategies.payment_strategy import PaymentContext

class TestPaymentContext(unittest.TestCase):

    @patch('strategies.payment_strategy.PaymentGatewayFactory.get_gateway')
    def test_execute_payment_with_razorpay(self, mock_get_gateway):
        mock_gateway = mock_get_gateway.return_value
        mock_gateway.process_payment = MagicMock()

        context = PaymentContext('razorpay')
        context.execute_payment(amount=500.00, currency='INR', receipt='order_rcptid_11')

        mock_gateway.process_payment.assert_called_once_with(
            amount=500.00,
            currency='INR',
            receipt='order_rcptid_11'
        )

    @patch('strategies.payment_strategy.PaymentGatewayFactory.get_gateway')
    def test_execute_payment_with_stripe(self, mock_get_gateway):
        mock_gateway = mock_get_gateway.return_value
        mock_gateway.process_payment = MagicMock()

        context = PaymentContext('stripe')
        context.execute_payment(amount=20.00, currency='usd', receipt='order_rcptid_22')

        mock_gateway.process_payment.assert_called_once_with(
            amount=20.00,
            currency='usd',
            receipt='order_rcptid_22',
            receipt_email=None
        )

    @patch('strategies.payment_strategy.PaymentGatewayFactory.get_gateway')
    def test_execute_payment_with_upi(self, mock_get_gateway):
        mock_gateway = mock_get_gateway.return_value
        mock_gateway.process_payment = MagicMock()

        context = PaymentContext('upi')
        context.execute_payment(amount=1000.00, currency='INR', receipt='order_rcptid_33')

        mock_gateway.process_payment.assert_called_once_with(
            amount=1000.00,
            currency='INR',
            receipt='order_rcptid_33'
        )

if __name__ == '__main__':
    unittest.main()


## 5. Running the Tests
Run the tests with:

In [ ]:
python -m unittest discover tests

## 6. Summary
Updated Design: We’ve integrated UPI into the existing payment gateway design using the same modular and design pattern-driven approach.
Robust Testing: Comprehensive test cases cover the new UPI functionality, ensuring that it integrates smoothly with the existing system.
Extensibility: The architecture is now even more extensible, allowing for further additions of payment methods with minimal changes to the overall structure.
This approach ensures that the system remains robust, scalable, and easy to maintain.

**********************************

**************************

To design a workflow for a payment service or gateway in Python, integrating the mentioned libraries and tools, we'll break it down into various components. This workflow will cover the entire lifecycle of a payment transaction, from API request to final confirmation, while ensuring security, scalability, and reliability. Below is the big picture of the architecture.

## 1. User Interaction Layer
`Frontend:` Users interact with the payment system through a web or mobile application.<br>
    `Technologies:` ReactJS, Angular, or any other frontend framework.<br>
`Request Flow:` Users initiate payment requests via a frontend interface, which sends the data to the backend API.

## 2. API Gateway Layer
`Flask/Django API:` The frontend communicates with the backend via a RESTful API developed using Flask or Django.
    `Endpoints:`<br>
        /payment/initiate: Initiates a payment request.<br>
        /payment/status: Checks the status of a payment.<br>
        /payment/callback: Handles payment gateway callbacks.<br>
## Security:
`JWT Authentication:` Secures API endpoints using PyJWT for token-based authentication.<br>
`Rate Limiting:` Implemented to prevent abuse.
## 3. Business Logic Layer
`Payment Processing:`<br>
    `Strategy Pattern:` Use the Strategy design pattern to support multiple payment gateways (e.g., Stripe, Razorpay).>br>
    `Libraries:`<br>
        `Stripe:` For Stripe payments.<br>
        `Razorpay:` For Razorpay payments.</br
        `PayPal:` For PayPal payments.<br>
    `Factory Pattern:` Use the Factory pattern to instantiate the appropriate payment gateway handler based on the user’s choice.<br>
`Error Handling:`<br>
    `Sentry:` Real-time error tracking and logging.
## 4. Asynchronous Processing Layer
`Task Queue:`<br>
`Celery:` Handles background tasks like confirming payments, sending emails, and updating the database.<br>
`Redis/RabbitMQ:` Acts as a message broker for Celery.<br>
`Event Streaming:`<br>
`Kafka:` Manages real-time events and logs payment requests and responses for later analysis and debugging.
## 5. Data Persistence Layer
`Database:`<br>
`PostgreSQL/MySQL:` Stores payment-related data, user information, and transaction history.<br>
`SQLAlchemy/Django ORM:` Manages database interactions, ensuring data consistency and normalization.<br>
`Schema Design:`<br>
`5th Normal Form (5NF):` The database schema is designed to adhere to 5NF, ensuring minimal redundancy and optimal performance.
## 6. Security Layer
`Encryption:`<br>
`cryptography:` Encrypts sensitive data such as credit card details.<br>
`Data Integrity:`<br>
`JWT and HMAC:` Ensure that data is not tampered with during transmission.<br>
`Secure Communication:`<br>
`HTTPS/TLS:` All communications between clients and servers are encrypted using HTTPS.
## 7. Monitoring and Logging Layer
Logging:
Loguru: Captures detailed logs of the application's behavior for troubleshooting.
Error Monitoring:
Sentry: Continuously monitors the application for errors and exceptions.
## 8. Deployment Layer
Containerization:
Docker: Containers package the application and its dependencies for consistent environments across development, testing, and production.
Orchestration:
Kubernetes: Manages the deployment, scaling, and monitoring of containers across multiple nodes.
CI/CD:
Jenkins/GitLab CI: Automates the building, testing, and deployment of the application.
## 9. Callback and Notification Layer
Webhook Handling:
Payment Gateway Callbacks: The system handles asynchronous notifications from payment gateways to update payment status.
Queue for Retry Logic: Failed callbacks are retried using a queue system.
## 10. Auditing and Compliance Layer
Logging of Transactions:
Kafka: Logs all transactions for audit trails and compliance checks.
Compliance:
PCI-DSS: The system is designed to comply with PCI-DSS standards for secure handling of payment information.
Workflow Diagram
User Initiates Payment: User selects a payment method on the frontend and submits payment details.
API Gateway Receives Request: Flask/Django API receives the payment request and authenticates the user using JWT.
Payment Strategy Selection: The system selects the appropriate payment gateway handler (e.g., Stripe, Razorpay) using the Strategy and Factory design patterns.
Payment Processing: The selected gateway processes the payment. The request may be synchronous or asynchronous.
Asynchronous Tasks: Celery handles background tasks such as sending confirmation emails and updating the database.
Callback Handling: The payment gateway sends a callback to the /payment/callback endpoint, which updates the payment status in the database.
Event Logging: Kafka logs all payment transactions for future analysis.
Data Encryption: Sensitive payment information is encrypted using the cryptography library.
Monitoring and Logging: Errors and logs are captured and monitored using Sentry and Loguru.
Deployment: The application is containerized with Docker and managed using Kubernetes for scalability.





## Workflow Diagram
User Initiates Payment: User selects a payment method on the frontend and submits payment details.
API Gateway Receives Request: Flask/Django API receives the payment request and authenticates the user using JWT.
Payment Strategy Selection: The system selects the appropriate payment gateway handler (e.g., Stripe, Razorpay) using the Strategy and Factory design patterns.
Payment Processing: The selected gateway processes the payment. The request may be synchronous or asynchronous.
Asynchronous Tasks: Celery handles background tasks such as sending confirmation emails and updating the database.
Callback Handling: The payment gateway sends a callback to the /payment/callback endpoint, which updates the payment status in the database.
Event Logging: Kafka logs all payment transactions for future analysis.
Data Encryption: Sensitive payment information is encrypted using the cryptography library.
Monitoring and Logging: Errors and logs are captured and monitored using Sentry and Loguru.
Deployment: The application is containerized with Docker and managed using Kubernetes for scalability.

<img src="architect_design.png" alt="architecture" width="500" height="1200">